In [16]:
!pip install nba_api
import json
import pandas
import requests
import nba_api
import pandas as pd

from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
response = shotchartdetail.ShotChartDetail(
	team_id=0,
	player_id=201935,
	season_nullable='2018-19',
	season_type_all_star='Regular Season'
)

content = json.loads(response.get_json())

In [18]:
all_players = players.get_players()
all_players_id = [p['id'] for p in all_players]

In [19]:
len(all_players_id)

4831

In [20]:
active_players = []
for p in all_players:
    if p['is_active'] == True:
        active_players.append(p['id'])
        
print(len(active_players))

582


In [22]:
season_years = [i for i in range(1990, 2022)]

In [23]:
import requests
import json


In [24]:
def get_season_no(season_year):
    assert 1990 <= season_year <= 2022
    return str(season_year) + '-' + str(season_year+1)[-2:]

In [25]:
get_season_no(2000)

'2000-01'

In [26]:
def get_data_player_year(player_id, season_year):
    url_base = 'https://stats.nba.com/stats/shotchartdetail'
    print("player_id", player_id)
    season = get_season_no(season_year)
    print("season", season)
    headers = {
            'Host': 'stats.nba.com',
            'Connection': 'keep-alive',
            'Accept': 'application/json, text/plain, */*',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
            'Referer': 'https://stats.nba.com/',
            "x-nba-stats-origin": "stats",
            "x-nba-stats-token": "true",
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-US,en;q=0.9',
        }
    parameters = {
        'ContextMeasure': 'FGA',
        'LastNGames': 0,
        'LeagueID': '00',
        'Month': 0,
        'OpponentTeamID': 0,
        'Period': 0,
        'PlayerID': player_id,
        'SeasonType': 'Regular Season',
        'TeamID': 0,
        'VsDivision': '',
        'VsConference': '',
        'SeasonSegment': '',
        'Season': season,
        'RookieYear': '',
        'PlayerPosition': '',
        'Outcome': '',
        'Location': '',
        'GameSegment': '',
        'GameId': '',
        'DateTo': '',
        'DateFrom': ''
    }
    response = requests.get(url_base, params=parameters, headers=headers)
    content = json.loads(response.content)
    # transform contents into dataframe
    results = content['resultSets'][0]
    headers = results['headers']
    rows = results['rowSet']
    df = pd.DataFrame(rows)
    df.columns = headers
    return df

In [27]:
def get_player_data(player_id):
    url_base = 'https://stats.nba.com/stats/commonplayerinfo?LeagueID=&PlayerID='
    print("player_id",player_id)
    headers = {
            'Host': 'stats.nba.com',
            'Connection': 'keep-alive',
            'Accept': 'application/json, text/plain, */*',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
            'Referer': 'https://stats.nba.com/',
            "x-nba-stats-origin": "stats",
            "x-nba-stats-token": "true",
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-US,en;q=0.9',
        }
    response = requests.get(url_base+str(player_id), headers=headers)
    content = json.loads(response.content)
    # transform contents into dataframe
    results = content['resultSets'][0]
    headers = results['headers']
    rows = results['rowSet']
    df = pd.DataFrame(rows)
    df.columns = headers
    return df

In [32]:
df = {}

import time
print("helo")
for i in range(2536,4832):
    player_id = all_players_id[i]
    try:
        player_info = get_player_data(player_id)
        from_year = player_info['FROM_YEAR'][0]
        to_year = player_info['TO_YEAR'][0]
    except:
        to_year = 2021
        from_year = 1990
    if to_year is None:
        to_year = 2021
    if from_year is None:
        from_year = 1990
    
    if to_year >= 1990:
        for season_year in range(max(from_year, 1990), min(to_year, 2021)+1):
            print(season_year)
            try:
                time.sleep(1)
                dft = get_data_player_year(player_id, season_year)
                dft.to_csv(str(player_id)+'-'+str(season_year)+'.csv', index=False)
                if season_year not in df:
                  df[season_year] = dft
                else:
                  df[season_year] = pd.concat([df[season_year], dft], axis=0)
                print('success')
            except:
                print('unsuccessful')
                pass

# write to csv file
# df.to_csv(str(parameters['PlayerID'])+'.csv', index=False)

helo
player is 202720
player_id 202720
here1
here2
here4
here5
2011
player_id 202720
season 2011-12
here1
here2
here3
here4
here5
dft done
success
2012
player_id 202720
season 2012-13
here1
here2
here3
here4
here5
dft done
success
2013
player_id 202720
season 2013-14
here1
here2
here3
here4
here5
dft done
success
2014
player_id 202720
season 2014-15
here1
here2
here3
here4
here5
dft done
success
2015
player_id 202720
season 2015-16
here1
here2
here3
here4
here5
dft done
success
2016
player_id 202720
season 2016-17
here1
here2
here3
here4
here5
dft done
success
2017
player_id 202720
season 2017-18
here1
here2
here3
here4
here5
dft done
success
2018
player_id 202720
season 2018-19
here1
here2
here3
here4
here5
dft done
success
player is 77375
player_id 77375
here1
here2
here4
here5
player is 77376
player_id 77376
here1
here2
here4
here5
1990
player_id 77376
season 1990-91
here1
here2
here3
here4
here5
unsuccessful
1991
player_id 77376
season 1991-92
here1
here2
here3
here4
here5
unsucces

here4
here5
2011
player_id 202732
season 2011-12
here1
here2
here3
here4
here5
dft done
check
success
2012
player_id 202732
season 2012-13
here1
here2
here3
here4
here5
dft done
check
success
2013
player_id 202732
season 2013-14
here1
here2
here3
here4
here5
dft done
check
success
2014
player_id 202732
season 2014-15
here1
here2
here3
here4
here5
unsuccessful
2015
player_id 202732
season 2015-16
here1
here2
here3
here4
here5
unsuccessful
2016
player_id 202732
season 2016-17
here1
here2
here3
here4
here5
dft done
check
success
2017
player_id 202732
season 2017-18
here1
here2
here3
here4
here5
dft done
check
success
player is 77388
player_id 77388
here1
here2
here4
here5
player is 203081
player_id 203081
here1
here2
here4
here5
2012
player_id 203081
season 2012-13
here1
here2
here3
here4
here5
dft done
check
success
2013
player_id 203081
season 2013-14
here1
here2
here3
here4
here5
dft done
check
success
2014
player_id 203081
season 2014-15
here1
here2
here3
here4
here5
dft done
check
su

player_id 38
season 1997-98
here1
here2
here3
here4
here5
dft done
check
success
player is 1609
player_id 1609
here1
here2
here4
here5
2000
player_id 1609
season 2000-01
here1
here2
here3
here4
here5
dft done
check
success
2001
player_id 1609
season 2001-02
here1
here2
here3
here4
here5
dft done
check
success
2002
player_id 1609
season 2002-03
here1
here2
here3
here4
here5
dft done
check
success
player is 3
player_id 3
here1
here2
here4
here5
1990
player_id 3
season 1990-91
here1
here2
here3
here4
here5
unsuccessful
1991
player_id 3
season 1991-92
here1
here2
here3
here4
here5
unsuccessful
1992
player_id 3
season 1992-93
here1
here2
here3
here4
here5
unsuccessful
1993
player_id 3
season 1993-94
here1
here2
here3
here4
here5
unsuccessful
1994
player_id 3
season 1994-95
here1
here2
here3
here4
here5
unsuccessful
1995
player_id 3
season 1995-96
here1
here2
here3
here4
here5
unsuccessful
1996
player_id 3
season 1996-97
here1
here2
here3
here4
here5
dft done
check
success
1997
player_id 3
s

player_id 201567
season 2011-12
here1
here2
here3
here4
here5
dft done
check
success
2012
player_id 201567
season 2012-13
here1
here2
here3
here4
here5
dft done
check
success
2013
player_id 201567
season 2013-14
here1
here2
here3
here4
here5
dft done
check
success
2014
player_id 201567
season 2014-15
here1
here2
here3
here4
here5
dft done
check
success
2015
player_id 201567
season 2015-16
here1
here2
here3
here4
here5
dft done
check
success
2016
player_id 201567
season 2016-17
here1
here2
here3
here4
here5
dft done
check
success
2017
player_id 201567
season 2017-18
here1
here2
here3
here4
here5
dft done
check
success
2018
player_id 201567
season 2018-19
here1
here2
here3
here4
here5
dft done
check
success
2019
player_id 201567
season 2019-20
here1
here2
here3
here4
here5
dft done
check
success
2020
player_id 201567
season 2020-21
here1
here2
here3
here4
here5
dft done
check
success
2021
player_id 201567
season 2021-22
here1
here2
here3
here4
here5
dft done
check
success
player is 77413

player_id 248
season 1999-00
here1
here2
here3
here4
here5
dft done
check
success
2000
player_id 248
season 2000-01
here1
here2
here3
here4
here5
dft done
check
success
2001
player_id 248
season 2001-02
here1
here2
here3
here4
here5
dft done
check
success
2002
player_id 248
season 2002-03
here1
here2
here3
here4
here5
dft done
check
success
2003
player_id 248
season 2003-04
here1
here2
here3
here4
here5
dft done
check
success
2004
player_id 248
season 2004-05
here1
here2
here3
here4
here5
dft done
check
success
player is 77426
player_id 77426
here1
here2
here4
here5
1991
player_id 77426
season 1991-92
here1
here2
here3
here4
here5
unsuccessful
1992
player_id 77426
season 1992-93
here1
here2
here3
here4
here5
unsuccessful
player is 77427
player_id 77427
here1
here2
here4
here5
player is 1627815
player_id 1627815
here1
here2
here4
here5
2016
player_id 1627815
season 2016-17
here1
here2
here3
here4
here5
dft done
check
success
player is 1928
player_id 1928
here1
here2
here4
here5
1999
pla

...

here4
here5
unsuccessful
1991
player_id 116
season 1991-92
here1
here2
here3
here4
here5
unsuccessful
1992
player_id 116
season 1992-93
here1
here2
here3
here4
here5
unsuccessful
1993
player_id 116
season 1993-94
here1
here2
here3
here4
here5
unsuccessful
1994
player_id 116
season 1994-95
here1
here2
here3
here4
here5
unsuccessful
1995
player_id 116
season 1995-96
here1
here2
here3
here4
here5
unsuccessful
1996
player_id 116
season 1996-97
here1
here2
here3
here4
here5
dft done
check
success
player is 78609
player_id 78609
here1
here2
here4
here5
player is 78611
player_id 78611
here1
here2
here4
here5
1990
player_id 78611
season 1990-91
here1
here2
here3
here4
here5
unsuccessful
player is 1630218
player_id 1630218
here1
here2
here4
here5
2020
player_id 1630218
season 2020-21
here1
here2
here3
here4
here5
dft done
check
success
2021
player_id 1630218
season 2021-22
here1
here2
here3
here4
here5
dft done
check
success
player is 2254
player_id 2254
here1
here2
here4
here5
2001
player_id 2

here4
here5
dft done
check
success
player is 203203
player_id 203203
here1
here2
here4
here5
2012
player_id 203203
season 2012-13
here1
here2
here3
here4
here5
dft done
check
success
player is 1626153
player_id 1626153
here1
here2
here4
here5
2015
player_id 1626153
season 2015-16
here1
here2
here3
here4
here5
dft done
check
success
2016
player_id 1626153
season 2016-17
here1
here2
here3
here4
here5
dft done
check
success
2017
player_id 1626153
season 2017-18
here1
here2
here3
here4
here5
dft done
check
success
2018
player_id 1626153
season 2018-19
here1
here2
here3
here4
here5
dft done
check
success
2019
player_id 1626153
season 2019-20
here1
here2
here3
here4
here5
dft done
check
success
2020
player_id 1626153
season 2020-21
here1
here2
here3
here4
here5
dft done
check
success
2021
player_id 1626153
season 2021-22
here1
here2
here3
here4
here5
dft done
check
success
player is 2748
player_id 2748
here1
here2
here4
here5
2004
player_id 2748
season 2004-05
here1
here2
here3
here4
here5
d

here4
here5
2014
player_id 203923
season 2014-15
here1
here2
here3
here4
here5
dft done
check
success
2015
player_id 203923
season 2015-16
here1
here2
here3
here4
here5
dft done
check
success
2016
player_id 203923
season 2016-17
here1
here2
here3
here4
here5
dft done
check
success
2017
player_id 203923
season 2017-18
here1
here2
here3
here4
here5
dft done
check
success
player is 1626202
player_id 1626202
here1
here2
here4
here5
2015
player_id 1626202
season 2015-16
here1
here2
here3
here4
here5
dft done
check
success
2016
player_id 1626202
season 2016-17
here1
here2
here3
here4
here5
dft done
check
success
2017
player_id 1626202
season 2017-18
here1
here2
here3
here4
here5
dft done
check
success
player is 1748
player_id 1748
here1
here2
here4
here5
1998
player_id 1748
season 1998-99
here1
here2
here3
here4
here5
dft done
check
success
player is 78634
player_id 78634
here1
here2
here4
here5
player is 201156
player_id 201156
here1
here2
here4
here5
2007
player_id 201156
season 2007-08
he

here4
here5
2016
player_id 1627835
season 2016-17
here1
here2
here3
here4
here5
dft done
check
success
2017
player_id 1627835
season 2017-18
here1
here2
here3
here4
here5
dft done
check
success
player is 1627790
player_id 1627790
here1
here2
here4
here5
2017
player_id 1627790
season 2017-18
here1
here2
here3
here4
here5
dft done
check
success
2018
player_id 1627790
season 2018-19
here1
here2
here3
here4
here5
dft done
check
success
2019
player_id 1627790
season 2019-20
here1
here2
here3
here4
here5
dft done
check
success
player is 78647
player_id 78647
here1
here2
here4
here5
player is 78648
player_id 78648
here1
here2
here4
here5
player is 1627826
player_id 1627826
here1
here2
here4
here5
2016
player_id 1627826
season 2016-17
here1
here2
here3
here4
here5
dft done
check
success
2017
player_id 1627826
season 2017-18
here1
here2
here3
here4
here5
dft done
check
success
2018
player_id 1627826
season 2018-19
here1
here2
here3
here4
here5
dft done
check
success
2019
player_id 1627826
seaso

IndexError: list index out of range